<a href="https://colab.research.google.com/github/ritzx21/Fake-News-Prediction/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake News Prediction using Python

About Dataset

* id : unique id for a news article
* title : the title of a news article
* author : author of the new article
* text : the text of the article (could be incomplete)
*Label : a label that marks whether the news article is real or fake.



```
 1 : Fake News
 0 : Real News
```

In [ ]:
import numpy as np
import pandas as pd
import re  #re - regular expression , useful for searching text data document
from nltk.corpus import stopwords  # nltk - natural Language toolkit
from nltk.stem.porter import PorterStemmer


stopwords - to words words from the text (dataset) which do not add much value . Words like the , a , where etc which can be omitted auring processing

PorterStemmer -gives us the root words for specific words.

Stemming - it is the process of reducing a word to it's root word. ex: actor , actress , acting -> act

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer  # helps convert the text into feature vector(numerical data)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
test_df = drive.mount('/content/gdrive')
#test_df = drive.mount('/fake_new_train/train.csv')

Mounted at /content/gdrive


In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [63]:
train_df = pd.read_csv("/content/gdrive/MyDrive/fake_news_train/train.csv")

### Data Pre-Processing

In [ ]:
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
train_df.shape  #20,800 rows and 5 columns

(20800, 5)

In [64]:
# null values
train_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [65]:
# replace the null values with empty string to remove all Null or NaN
train_df = train_df.fillna(" ")

In [66]:
train_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
# Merge the author name and news title which will be then used for the prediction
train_df['content'] = train_df["author"] + ' '+ train_df["title"]

# Combine the author and title from dataset and save it in 'content' column

In [ ]:
train_df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
# separating the label from the dataset , cuz we need to predict the lable hence we can't have the label column in hte data while training

X = train_df.drop(columns = 'label', axis = 1)
Y = train_df['label']

In [ ]:
X

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [ ]:
Y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [ ]:
# Stemming
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

Here we are creating a function called stemming. The content is the text input that the function is going to process

Initially we are calling the re( regular expression) library to search for a  x in the content.

Then used the function 'sub' it basically means that we can substitute certain values. Since we just want all the words and the alphabets and not the numbers we use the `[^a-zA-Z]` That line of code excludes everything which is not alphabet from small a to small z and capital A to capital Z

`re.sub('[a-zA-Z]', ' ',content)` -

`'[a-zA-Z]', ' ', 'content`This part means that anything in the text that isn't an alphabet will be replaced by ' ' (an empty string) in the content column

then we need to convert all these words and letters to lowercase , then the words are split and converted to a list.

Then a for loop is used to remove all stopwords from the content

Next join all the words in the list and return
Next we need to apply this function to the content column

In [ ]:
train_df['content'] = train_df['content'].apply(stemming)

In [ ]:
print(train_df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
train_df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...


In [ ]:
X = train_df['content'].values
Y = train_df['label'].values

In [ ]:
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [ ]:
Y

array([1, 0, 1, ..., 0, 1, 1])

In [ ]:
Y.shape

(20800,)

In [ ]:
X.shape

(20800,)

In [ ]:
#Convert textual data to numerical data
vectorizer = TfidfVectorizer() #Tf - term frequency , idf - inverse data frequency
vectorizer.fit(X)

X = vectorizer.transform(X)

### Splitting data into training and testing data

In [ ]:
X_train , X_test , Y_train , Y_test = train_test_split(X,Y , test_size=0.2,stratify = Y ,random_state = 42)

Training the Model : Logistic Regression

In [98]:
model = LogisticRegression()

In [ ]:
model.fit(X_train , Y_train)

LogisticRegression()

### Evaluation

In [ ]:
X_predict = model.predict(X_train)
accuracy = accuracy_score(X_predict,Y_train)
accuracy

0.9874399038461539

In [ ]:
print("Accuracy Score fo Training Data : ",accuracy)

Accuracy Score fo Training Data :  0.9874399038461539


Test data using the logistic regression model

In [ ]:
X_test_predict = model.predict(X_test)
test_accuracy = accuracy_score(X_test_predict , Y_test)

In [ ]:
print("The accuracy score of the Testing Data : ",test_accuracy)

The accuracy score of the Testing Data :  0.9752403846153846


Making a Predictive System

In [ ]:
X_new = X_test[10]

predict = model.predict(X_new)
print(predict)

if(predict[0]==0):
  print("The news is Real")
else:
  print("The news is Fake")

[1]
The news is Fake


In [ ]:
print(Y_test[10])

1


### Testing on Validation dataset



Loading Dataset

In [67]:
#loading validation set
valid_df = pd.read_csv("https://raw.githubusercontent.com/ritzx21/Fake-News-Prediction/main/test.csv")
submit_df = pd.read_csv("https://raw.githubusercontent.com/ritzx21/Fake-News-Prediction/main/submit.csv")

In [ ]:
submit_df.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [49]:
valid_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


Null values

In [68]:
#checking for null values
valid_df.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

In [69]:
valid_df = valid_df.fillna(" ")

In [70]:
#removed null values
valid_df.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

Content column

In [71]:
valid_df['content'] = valid_df['author'] + valid_df['title']

In [72]:
valid_df.head()

,id,title,author,text,content
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",David StreitfeldSpecter of Trump Loosens Tongu...
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists n...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,Common Dreams#NoDAPL: Native American Leaders ...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",Daniel VictorTim Tebow Will Attempt Another Co...
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,Truth Broadcast NetworkKeiser Report: Meme War...


Stemming

In [74]:
valid_df['content'] = valid_df['content'].apply(stemming)

In [78]:
valid_df['content']

0       david streitfeldspect trump loosen tongu purs ...
1       russian warship readi strike terrorist near al...
2       common dream nodapl nativ american leader vow ...
3       daniel victortim tebow attempt anoth comeback ...
4           truth broadcast networkkeis report meme war e
                              ...                        
5195    jodi rosenth bangladeshi traffic jam never end...
5196    sheryl gay stolbergjohn kasich sign one abort ...
5197    mike mcphatecalifornia today exactli sushi new...
5198                us marin deploy russian border norway
5199        teddi wayneawkward sex onscreen new york time
Name: content, Length: 5200, dtype: object

In [93]:
X_valid = valid_df['content'].values
Y_valid = submit_df['label'].values

In [94]:
X_valid

array(['david streitfeldspect trump loosen tongu purs string silicon valley new york time',
       'russian warship readi strike terrorist near aleppo',
       'common dream nodapl nativ american leader vow stay winter file lawsuit polic',
       ..., 'mike mcphatecalifornia today exactli sushi new york time',
       'us marin deploy russian border norway',
       'teddi wayneawkward sex onscreen new york time'], dtype=object)

In [99]:
Y_valid

array([0, 1, 0, ..., 0, 1, 0])

Vectornizer

In [95]:
vector = TfidfVectorizer()

In [96]:
vector.fit(X_valid)

TfidfVectorizer()

In [97]:
X_valid = vectorizer.transform(X_valid)

fitting the model

In [101]:
model.fit(X_valid, Y_valid)

LogisticRegression()

In [103]:
X_valid_predict = model.predict(X_valid)
accuracy = accuracy_score(X_valid_predict , Y_valid)

In [104]:
print("The accuracy on the validation set is : ",accuracy)

The accuracy on the validation set is :  0.8253846153846154
